In [ ]:
#Who is the best basketball player of all time? I will be looking at diving into
#the analytics of the Goat debate, to decide if Michael Jordan or Lebron James is the 
#greatest basketball player ever. I will be analyzing what factors into that distinction, including
#advanced stats, longevity, winning metrics, and team analytics.
#I used the NBA_api to pull the data

In [64]:
#Write a csv file of the Combined dataframe
combined_df.to_csv('goat_data.csv', index=False)

In [ ]:
result_df = combined_df.groupby(['SEASON_ID', 'Player_ID']).agg({'REB': 'sum', 'PTS': 'sum','Game_ID': 'count'}).reset_index()
result_df['Games_Played'] = result_df['Game_ID']
result_df = result_df.drop('Game_ID', axis=1)
result_df['id'] = result_df['Player_ID']
result_df = result_df.drop('Player_ID', axis=1)
result_df['PPG'] = result_df['PTS']/result_df['Games_Played']
result_df = pd.merge(result_df, player_dict, on='id', how='left')
result_df['Season'] = result_df.groupby('full_name')['SEASON_ID'].rank(ascending=False).astype(int)
result_df

In [60]:
#Create a Plot showing the trend in Points per game for each season of the player's career
fig = px.line(result_df, x='Season', y='PPG', color='full_name', title='PPG for Each Season of Career',
                 labels={'PPG': 'Points Per Game', 'Season': 'Season', 'full_name':'Name'},
                 hover_data=['full_name'])

fig.show()
plotly.offline.plot(fig, filename='ppg_plot.html')


'ppg_plot.html'

In [430]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml as lx
import re

In [431]:
#Michael Jordan Regular Season Advanced Stats
dates = []
usages = []
true_shoot = []
efg = []
trb = []
ast_pct = []
ortg = []
drtg = []
gmsc = []
playoffs = []
season = []
for year in range(1985,2004):
    if year in [1994, 1999, 2000, 2001]:
        continue
    else:   
        url = f'https://www.basketball-reference.com/players/j/jordami01/gamelog-advanced/{year}/'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        table1 = soup.find('table', id='pgl_advanced')
        for nba_data in table1.find_all('tbody'):
            rows = nba_data.find_all('tr')
        for row in rows:
            date = row.find_all('td', attrs={"data-stat": "date_game"})
            usg = row.find_all('td', attrs={"data-stat": "usg_pct"})
            true = row.find_all('td', attrs={"data-stat": "ts_pct"})
            effect = row.find_all('td', attrs={"data-stat": "efg_pct"})
            board = row.find_all('td', attrs={"data-stat": "trb_pct"})
            ast = row.find_all('td', attrs={"data-stat": "ast_pct"})
            off = row.find_all('td', attrs={"data-stat": "off_rtg"})
            defense = row.find_all('td', attrs={"data-stat": "def_rtg"})
            gs = row.find_all('td', attrs={"data-stat": "game_score"})
            dates.append(date)
            usages.append(usg)
            true_shoot.append(true)
            efg.append(effect)
            trb.append(board)
            ast_pct.append(ast)
            ortg.append(off)
            drtg.append(defense)
            gmsc.append(gs)
            playoffs.append(False)
            season.append(f'{year-1}-{year}')
df = pd.DataFrame({'Season': season,'Date': dates, 'Usage %': usages,'True Shooting %': true_shoot,'Effective Fg %': efg, 'Total Rebound %': trb, 'Assist %' : ast_pct,'Offensive Rating': ortg, 'Defensive Rating' : drtg,'Game Score': gmsc,'Playoffs': playoffs})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Usage %'] = df['Usage %'].str.extract(r'>([\d.]+)<').astype(float)
df['True Shooting %'] = df['True Shooting %'].str.extract(r'>([\d.]+)<').astype(float)
df['Effective Fg %'] = df['Effective Fg %'].str.extract(r'>([\d.]+)<').astype(float)
df['Total Rebound %'] = df['Total Rebound %'].str.extract(r'>([\d.]+)<').astype(float)
df['Assist %'] = df['Assist %'].str.extract(r'>([\d.]+)<').astype(float)
df['Offensive Rating'] = df['Offensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Defensive Rating'] = df['Defensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Game Score'] = df['Game Score'].str.extract(r'>([\d.]+)<').astype(float)
df = df.dropna(subset=['Date'])
df['Player'] = 'Michael Jordan'
mj_reg_adv = df

In [411]:
#Michael Jordan Playoff Advanced Stats
dates = []
usages = []
true_shoot = []
efg = []
trb = []
ast_pct = []
ortg = []
drtg = []
gmsc = []
playoffs = []  
url = 'https://www.basketball-reference.com/players/j/jordami01/gamelog-playoffs-advanced/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table1 = soup.find('table', id='pgl_advanced_playoffs')
for nba_data in table1.find_all('tbody'):
    rows = nba_data.find_all('tr')
for row in rows:
    date = row.find_all('td', attrs={"data-stat": "date_game"})
    usg = row.find_all('td', attrs={"data-stat": "usg_pct"})
    true = row.find_all('td', attrs={"data-stat": "ts_pct"})
    effect = row.find_all('td', attrs={"data-stat": "efg_pct"})
    board = row.find_all('td', attrs={"data-stat": "trb_pct"})
    ast = row.find_all('td', attrs={"data-stat": "ast_pct"})
    off = row.find_all('td', attrs={"data-stat": "off_rtg"})
    defense = row.find_all('td', attrs={"data-stat": "def_rtg"})
    gs = row.find_all('td', attrs={"data-stat": "game_score"})
    dates.append(date)
    usages.append(usg)
    true_shoot.append(true)
    efg.append(effect)
    trb.append(board)
    ast_pct.append(ast)
    ortg.append(off)
    drtg.append(defense)
    gmsc.append(gs)
    playoffs.append(True)
df = pd.DataFrame({'Date': dates, 'Usage %': usages,'True Shooting %': true_shoot,'Effective Fg %': efg, 'Total Rebound %': trb, 'Assist %' : ast_pct,'Offensive Rating': ortg, 'Defensive Rating' : drtg,'Game Score': gmsc,'Playoffs': playoffs})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Usage %'] = df['Usage %'].str.extract(r'>([\d.]+)<').astype(float)
df['True Shooting %'] = df['True Shooting %'].str.extract(r'>([\d.]+)<').astype(float)
df['Effective Fg %'] = df['Effective Fg %'].str.extract(r'>([\d.]+)<').astype(float)
df['Total Rebound %'] = df['Total Rebound %'].str.extract(r'>([\d.]+)<').astype(float)
df['Assist %'] = df['Assist %'].str.extract(r'>([\d.]+)<').astype(float)
df['Offensive Rating'] = df['Offensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Defensive Rating'] = df['Defensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Game Score'] = df['Game Score'].str.extract(r'>([\d.]+)<').astype(float)
df = df.dropna(subset=['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = (df['Date'].dt.year - 1).astype(int).astype(str) + '-' + df['Date'].dt.year.astype(int).astype(str)
df['Player'] = 'Michael Jordan'
mj_play_adv = df

In [432]:
mj_play_adv

,Date,Usage %,True Shooting %,Effective Fg %,Total Rebound %,Assist %,Offensive Rating,Defensive Rating,Game Score,Playoffs,Season,Player
0,1985-04-19,23.0,0.491,0.368,5.6,30.8,121.0,110.0,21.5,True,1984-1985,Michael Jordan
1,1985-04-21,27.7,0.673,0.559,6.0,41.9,121.0,129.0,24.3,True,1984-1985,Michael Jordan
2,1985-04-24,36.0,0.530,0.462,12.4,29.3,118.0,106.0,28.2,True,1984-1985,Michael Jordan
3,1985-04-26,28.9,0.585,0.375,9.6,21.1,120.0,104.0,24.3,True,1984-1985,Michael Jordan
5,1986-04-17,47.0,0.575,0.500,6.3,10.2,110.0,124.0,29.3,True,1985-1986,Michael Jordan
...,...,...,...,...,...,...,...,...,...,...,...,...
210,1998-06-05,45.4,0.495,0.424,8.1,20.9,122.0,113.0,24.8,True,1997-1998,Michael Jordan
211,1998-06-07,29.9,0.637,0.500,5.1,11.3,126.0,65.0,18.6,True,1997-1998,Michael Jordan
212,1998-06-10,41.0,0.506,0.444,10.6,16.4,108.0,101.0,21.5,True,1997-1998,Michael Jordan
213,1998-06-12,37.2,0.454,0.346,6.1,22.0,97.0,104.0,14.8,True,1997-1998,Michael Jordan


In [433]:
#Michael Jordan Playoff Basic Stats
own_team = []
game_location = []
opposing_team = []
game_date = []
game_result = []
mins = []
field_goal = []
field_goal_att = []
field_goal_pct = []
three_field_goal = []
three_field_goal_att = []
three_field_goal_pct = []
free_throw = []
free_throw_att = []
free_throw_pct = []
rebounds = []
assist = []
steal = []
block = []
turnover = []
points = []
url = 'https://www.basketball-reference.com/players/j/jordami01/gamelog-playoffs/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table1 = soup.find('table', id='pgl_basic_playoffs')
for nba_data in table1.find_all('tbody'):
    rows = nba_data.find_all('tr')
for row in rows:
    team = row.find_all('td', attrs={"data-stat": "team_id"})
    location = row.find_all('td', attrs={"data-stat": "game_location"})
    opp_team = row.find_all('td', attrs={"data-stat": "opp_id"})
    date = row.find_all('td', attrs={"data-stat": "date_game"})
    result = row.find_all('td', attrs={"data-stat": "game_result"})
    min = row.find_all('td', attrs={"data-stat": "mp"})
    fg = row.find_all('td', attrs={"data-stat": "fg"})
    fga = row.find_all('td', attrs={"data-stat": "fga"})
    fg_pct = row.find_all('td', attrs={"data-stat": "fg_pct"})
    fg3 = row.find_all('td', attrs={"data-stat": "fg3"})
    fg3a = row.find_all('td', attrs={"data-stat": "fg3a"})
    fg3_pct = row.find_all('td', attrs={"data-stat": "fg3_pct"})
    ft = row.find_all('td', attrs={"data-stat": "ft"})
    fta = row.find_all('td', attrs={"data-stat": "fta"})
    ft_pct = row.find_all('td', attrs={"data-stat": "ft_pct"})
    trb = row.find_all('td', attrs={"data-stat": "trb"})
    ast = row.find_all('td', attrs={"data-stat": "ast"})
    stl = row.find_all('td', attrs={"data-stat": "stl"})
    blk = row.find_all('td', attrs={"data-stat": "blk"})
    tov = row.find_all('td', attrs={"data-stat": "tov"})
    pts = row.find_all('td', attrs={"data-stat": "pts"})
    own_team.append(team)
    game_location.append(location)
    opposing_team.append(opp_team)
    game_date.append(date)
    game_result.append(result)
    mins.append(min)
    field_goal.append(fg)
    field_goal_att.append(fga)
    field_goal_pct.append(fg_pct)
    three_field_goal.append(fg3)
    three_field_goal_att.append(fg3a)
    three_field_goal_pct.append(fg3_pct)
    free_throw.append(ft)
    free_throw_att.append(fta)
    free_throw_pct.append(ft_pct)
    rebounds.append(trb)
    assist.append(ast)
    steal.append(stl)
    block.append(blk)
    turnover.append(tov)
    points.append(pts)
    playoffs.append(True)
df = pd.DataFrame({
    'Team': own_team,
    'Location': game_location,
    'Opp Team': opposing_team,
    'Date': game_date,
    'Result': game_result,
    'Mins Played': mins,
    'Field Goal': field_goal,
    'Field Goal Att': field_goal_att,
    'Field Goal %': field_goal_pct,
    '3 Point Field Goal': three_field_goal,
    '3 Point Field Goal Att': three_field_goal_att,
    '3 Point Field Goal %': three_field_goal_pct,
    'Free Throw': free_throw,
    'Free Throw Att': free_throw_att,
    'Free Throw %': free_throw_pct,
    'Rebounds': rebounds,
    'Assists': assist,
    'Steals': steal,
    'Blocks': block,
    'Turnovers': turnover,
    'Points': points
})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Field Goal'] = df['Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal Att'] = df['Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal %'] = df['Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal'] = df['3 Point Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal Att'] = df['3 Point Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal %'] = df['3 Point Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw'] = df['Free Throw'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw Att'] = df['Free Throw Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw %'] = df['Free Throw %'].str.extract(r'>([\d.]+)<').astype(float)
df['Rebounds'] = df['Rebounds'].str.extract(r'>([\d.]+)<').astype(float)
df['Assists'] = df['Assists'].str.extract(r'>([\d.]+)<').astype(float)
df['Steals'] = df['Steals'].str.extract(r'>([\d.]+)<').astype(float)
df['Blocks'] = df['Blocks'].str.extract(r'>([\d.]+)<').astype(float)
df['Turnovers'] = df['Turnovers'].str.extract(r'>([\d.]+)<').astype(float)
df['Points'] = df['Points'].str.extract(r'>([\d.]+)<').astype(float)
df['Team'] = df['Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Location'] = df['Location'].str.extract(r'>([^<>]+)<')
df['Opp Team'] = df['Opp Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Result'] = df['Result'].str.extract(r'>([^<>]+)<')
df['Result'] = df['Result'].astype(str).str.split().str[0]
df['Mins Played'] = df['Mins Played'].str.extract(r'>(\d+:\d+)<')
df = df.dropna(subset=['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = (df['Date'].dt.year - 1).astype(int).astype(str) + '-' + df['Date'].dt.year.astype(int).astype(str)
df['Player'] = 'Michael Jordan'
mj_play_basic = df


In [434]:
mj_play_basic

,Team,Location,Opp Team,Date,Result,Mins Played,Field Goal,Field Goal Att,Field Goal %,3 Point Field Goal,3 Point Field Goal Att,3 Point Field Goal %,Free Throw,Free Throw Att,Free Throw %,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Season,Player
0,CHI,@,MIL,1985-04-19,L,44:00,7.0,19.0,0.368,0.0,1.0,0.000,9.0,10.0,0.900,4.0,10.0,3.0,0.0,1.0,23.0,1984-1985,Michael Jordan
1,CHI,@,MIL,1985-04-21,L,43:00,9.0,17.0,0.529,1.0,4.0,0.250,11.0,12.0,0.917,4.0,12.0,2.0,0.0,7.0,30.0,1984-1985,Michael Jordan
2,CHI,NaN,MIL,1985-04-24,W,42:00,12.0,26.0,0.462,0.0,1.0,0.000,11.0,16.0,0.688,8.0,7.0,4.0,1.0,3.0,35.0,1984-1985,Michael Jordan
3,CHI,NaN,MIL,1985-04-26,L,42:00,6.0,16.0,0.375,0.0,2.0,0.000,17.0,20.0,0.850,7.0,5.0,2.0,3.0,4.0,29.0,1984-1985,Michael Jordan
5,CHI,@,BOS,1986-04-17,L,43:00,18.0,36.0,0.500,0.0,0.0,NaN,13.0,15.0,0.867,4.0,2.0,2.0,1.0,5.0,49.0,1985-1986,Michael Jordan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,CHI,@,UTA,1998-06-05,W,40:14,14.0,33.0,0.424,0.0,2.0,0.000,9.0,10.0,0.900,5.0,3.0,1.0,1.0,0.0,37.0,1997-1998,Michael Jordan
211,CHI,NaN,UTA,1998-06-07,W,32:06,7.0,14.0,0.500,0.0,0.0,NaN,10.0,11.0,0.909,3.0,2.0,1.0,1.0,2.0,24.0,1997-1998,Michael Jordan
212,CHI,NaN,UTA,1998-06-10,W,43:10,12.0,27.0,0.444,0.0,0.0,NaN,10.0,15.0,0.667,8.0,2.0,2.0,0.0,3.0,34.0,1997-1998,Michael Jordan
213,CHI,NaN,UTA,1998-06-12,L,44:43,9.0,26.0,0.346,0.0,2.0,0.000,10.0,11.0,0.909,4.0,4.0,3.0,0.0,4.0,28.0,1997-1998,Michael Jordan


In [435]:
#Michael Jordan Regular Season Basic Stats
own_team = []
game_location = []
opposing_team = []
game_date = []
game_result = []
mins = []
field_goal = []
field_goal_att = []
field_goal_pct = []
three_field_goal = []
three_field_goal_att = []
three_field_goal_pct = []
free_throw = []
free_throw_att = []
free_throw_pct = []
rebounds = []
assist = []
steal = []
block = []
turnover = []
points = []
season = []
for year in range(1985,2004):
    if year in [1994, 1999, 2000, 2001]:
        continue
    else:   
        url = f'https://www.basketball-reference.com/players/j/jordami01/gamelog/{year}'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        table1 = soup.find('table', id='pgl_basic')
        for nba_data in table1.find_all('tbody'):
            rows = nba_data.find_all('tr')
        for row in rows:
            team = row.find_all('td', attrs={"data-stat": "team_id"})
            location = row.find_all('td', attrs={"data-stat": "game_location"})
            opp_team = row.find_all('td', attrs={"data-stat": "opp_id"})
            date = row.find_all('td', attrs={"data-stat": "date_game"})
            result = row.find_all('td', attrs={"data-stat": "game_result"})
            min = row.find_all('td', attrs={"data-stat": "mp"})
            fg = row.find_all('td', attrs={"data-stat": "fg"})
            fga = row.find_all('td', attrs={"data-stat": "fga"})
            fg_pct = row.find_all('td', attrs={"data-stat": "fg_pct"})
            fg3 = row.find_all('td', attrs={"data-stat": "fg3"})
            fg3a = row.find_all('td', attrs={"data-stat": "fg3a"})
            fg3_pct = row.find_all('td', attrs={"data-stat": "fg3_pct"})
            ft = row.find_all('td', attrs={"data-stat": "ft"})
            fta = row.find_all('td', attrs={"data-stat": "fta"})
            ft_pct = row.find_all('td', attrs={"data-stat": "ft_pct"})
            trb = row.find_all('td', attrs={"data-stat": "trb"})
            ast = row.find_all('td', attrs={"data-stat": "ast"})
            stl = row.find_all('td', attrs={"data-stat": "stl"})
            blk = row.find_all('td', attrs={"data-stat": "blk"})
            tov = row.find_all('td', attrs={"data-stat": "tov"})
            pts = row.find_all('td', attrs={"data-stat": "pts"})
            own_team.append(team)
            game_location.append(location)
            opposing_team.append(opp_team)
            game_date.append(date)
            game_result.append(result)
            mins.append(min)
            field_goal.append(fg)
            field_goal_att.append(fga)
            field_goal_pct.append(fg_pct)
            three_field_goal.append(fg3)
            three_field_goal_att.append(fg3a)
            three_field_goal_pct.append(fg3_pct)
            free_throw.append(ft)
            free_throw_att.append(fta)
            free_throw_pct.append(ft_pct)
            rebounds.append(trb)
            assist.append(ast)
            steal.append(stl)
            block.append(blk)
            turnover.append(tov)
            points.append(pts)
            playoffs.append(False)
            season.append(f'{year-1}-{year}')
df = pd.DataFrame({
    'Season': season,
    'Team': own_team,
    'Location': game_location,
    'Opp Team': opposing_team,
    'Date': game_date,
    'Result': game_result,
    'Mins Played': mins,
    'Field Goal': field_goal,
    'Field Goal Att': field_goal_att,
    'Field Goal %': field_goal_pct,
    '3 Point Field Goal': three_field_goal,
    '3 Point Field Goal Att': three_field_goal_att,
    '3 Point Field Goal %': three_field_goal_pct,
    'Free Throw': free_throw,
    'Free Throw Att': free_throw_att,
    'Free Throw %': free_throw_pct,
    'Rebounds': rebounds,
    'Assists': assist,
    'Steals': steal,
    'Blocks': block,
    'Turnovers': turnover,
    'Points': points
})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Field Goal'] = df['Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal Att'] = df['Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal %'] = df['Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal'] = df['3 Point Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal Att'] = df['3 Point Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal %'] = df['3 Point Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw'] = df['Free Throw'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw Att'] = df['Free Throw Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw %'] = df['Free Throw %'].str.extract(r'>([\d.]+)<').astype(float)
df['Rebounds'] = df['Rebounds'].str.extract(r'>([\d.]+)<').astype(float)
df['Assists'] = df['Assists'].str.extract(r'>([\d.]+)<').astype(float)
df['Steals'] = df['Steals'].str.extract(r'>([\d.]+)<').astype(float)
df['Blocks'] = df['Blocks'].str.extract(r'>([\d.]+)<').astype(float)
df['Turnovers'] = df['Turnovers'].str.extract(r'>([\d.]+)<').astype(float)
df['Points'] = df['Points'].str.extract(r'>([\d.]+)<').astype(float)
df['Team'] = df['Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Location'] = df['Location'].str.extract(r'>([^<>]+)<')
df['Opp Team'] = df['Opp Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Result'] = df['Result'].str.extract(r'>([^<>]+)<')
df['Result'] = df['Result'].astype(str).str.split().str[0]
df['Mins Played'] = df['Mins Played'].str.extract(r'>(\d+:\d+)<')
df = df.dropna(subset=['Date'])
df['Player'] = 'Michael Jordan'
mj_reg_basic = df


In [437]:
mj_playoffs = pd.merge(mj_play_basic, mj_play_adv, on=['Date', 'Player','Season'])
mj_reg_season = pd.merge(mj_reg_basic, mj_reg_adv, on=['Date', 'Player','Season'])
mj = pd.concat([mj_reg_season, mj_playoffs], ignore_index=True)
mj['Date'] = pd.to_datetime(mj['Date'])
player_column = mj.pop('Player')
mj.insert(0, 'Player', player_column)

In [438]:
mj

,Player,Season,Team,Location,Opp Team,Date,Result,Mins Played,Field Goal,Field Goal Att,Field Goal %,3 Point Field Goal,3 Point Field Goal Att,3 Point Field Goal %,Free Throw,Free Throw Att,Free Throw %,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Usage %,True Shooting %,Effective Fg %,Total Rebound %,Assist %,Offensive Rating,Defensive Rating,Game Score,Playoffs
0,Michael Jordan,1984-1985,CHI,NaN,WSB,1984-10-26,W,40:00,5.0,16.0,0.313,0.0,0.0,NaN,6.0,7.0,0.857,6.0,7.0,2.0,4.0,5.0,16.0,24.4,0.419,0.313,7.8,20.0,86.0,85.0,12.5,False
1,Michael Jordan,1984-1985,CHI,@,MIL,1984-10-27,L,34:00,8.0,13.0,0.615,0.0,0.0,NaN,5.0,5.0,1.000,5.0,5.0,2.0,1.0,3.0,21.0,20.3,0.691,0.615,7.4,24.6,130.0,107.0,19.4,False
2,Michael Jordan,1984-1985,CHI,NaN,MIL,1984-10-29,W,34:00,13.0,24.0,0.542,0.0,0.0,NaN,11.0,13.0,0.846,4.0,5.0,6.0,2.0,3.0,37.0,41.6,0.622,0.542,8.0,34.2,126.0,97.0,32.9,False
3,Michael Jordan,1984-1985,CHI,@,KCK,1984-10-30,W,36:00,8.0,21.0,0.381,0.0,0.0,NaN,9.0,9.0,1.000,4.0,5.0,3.0,1.0,6.0,25.0,35.8,0.501,0.381,7.4,22.0,101.0,100.0,14.7,False
4,Michael Jordan,1984-1985,CHI,@,DEN,1984-11-01,L,33:00,7.0,15.0,0.467,0.0,0.0,NaN,3.0,4.0,0.750,5.0,5.0,1.0,1.0,2.0,17.0,21.0,0.507,0.467,8.1,25.2,116.0,120.0,13.2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,Michael Jordan,1997-1998,CHI,@,UTA,1998-06-05,W,40:14,14.0,33.0,0.424,0.0,2.0,0.000,9.0,10.0,0.900,5.0,3.0,1.0,1.0,0.0,37.0,45.4,0.495,0.424,8.1,20.9,122.0,113.0,24.8,True
1261,Michael Jordan,1997-1998,CHI,NaN,UTA,1998-06-07,W,32:06,7.0,14.0,0.500,0.0,0.0,NaN,10.0,11.0,0.909,3.0,2.0,1.0,1.0,2.0,24.0,29.9,0.637,0.500,5.1,11.3,126.0,65.0,18.6,True
1262,Michael Jordan,1997-1998,CHI,NaN,UTA,1998-06-10,W,43:10,12.0,27.0,0.444,0.0,0.0,NaN,10.0,15.0,0.667,8.0,2.0,2.0,0.0,3.0,34.0,41.0,0.506,0.444,10.6,16.4,108.0,101.0,21.5,True
1263,Michael Jordan,1997-1998,CHI,NaN,UTA,1998-06-12,L,44:43,9.0,26.0,0.346,0.0,2.0,0.000,10.0,11.0,0.909,4.0,4.0,3.0,0.0,4.0,28.0,37.2,0.454,0.346,6.1,22.0,97.0,104.0,14.8,True


In [157]:
#Different Visualizations
#Comparion of Points per game over each season
#Usage Rate in Playoffs vs Regular Season by Season
#Win Shares in Playoffs vs Regular Season by Season
#Vorp by season with career vorp lines


[]

In [439]:
# Lebron James Regular Season Advanced Stats
dates = []
usages = []
true_shoot = []
efg = []
trb = []
ast_pct = []
ortg = []
drtg = []
gmsc = []
playoffs = []
season = []
for year in range(2004,2025):  
    url = f'https://www.basketball-reference.com/players/j/jamesle01/gamelog-advanced/{year}/'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table1 = soup.find('table', id='pgl_advanced')
    for nba_data in table1.find_all('tbody'):
        rows = nba_data.find_all('tr')
    for row in rows:
        date = row.find_all('td', attrs={"data-stat": "date_game"})
        usg = row.find_all('td', attrs={"data-stat": "usg_pct"})
        true = row.find_all('td', attrs={"data-stat": "ts_pct"})
        effect = row.find_all('td', attrs={"data-stat": "efg_pct"})
        board = row.find_all('td', attrs={"data-stat": "trb_pct"})
        ast = row.find_all('td', attrs={"data-stat": "ast_pct"})
        off = row.find_all('td', attrs={"data-stat": "off_rtg"})
        defense = row.find_all('td', attrs={"data-stat": "def_rtg"})
        gs = row.find_all('td', attrs={"data-stat": "game_score"})
        dates.append(date)
        usages.append(usg)
        true_shoot.append(true)
        efg.append(effect)
        trb.append(board)
        ast_pct.append(ast)
        ortg.append(off)
        drtg.append(defense)
        gmsc.append(gs)
        playoffs.append(False)
        season.append(f'{year-1}-{year}')
df = pd.DataFrame({'Season': season,'Date': dates, 'Usage %': usages,'True Shooting %': true_shoot,'Effective Fg %': efg, 'Total Rebound %': trb, 'Assist %' : ast_pct,'Offensive Rating': ortg, 'Defensive Rating' : drtg,'Game Score': gmsc,'Playoffs': playoffs})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Usage %'] = df['Usage %'].str.extract(r'>([\d.]+)<').astype(float)
df['True Shooting %'] = df['True Shooting %'].str.extract(r'>([\d.]+)<').astype(float)
df['Effective Fg %'] = df['Effective Fg %'].str.extract(r'>([\d.]+)<').astype(float)
df['Total Rebound %'] = df['Total Rebound %'].str.extract(r'>([\d.]+)<').astype(float)
df['Assist %'] = df['Assist %'].str.extract(r'>([\d.]+)<').astype(float)
df['Offensive Rating'] = df['Offensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Defensive Rating'] = df['Defensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Game Score'] = df['Game Score'].str.extract(r'>([\d.]+)<').astype(float)
df = df.dropna(subset=['Date'])
df['Player'] = 'Lebron James'
lbj_reg_adv = df


In [440]:
lbj_reg_adv

,Season,Date,Usage %,True Shooting %,Effective Fg %,Total Rebound %,Assist %,Offensive Rating,Defensive Rating,Game Score,Playoffs,Player
0,2003-2004,2003-10-29,25.6,0.586,0.600,8.6,39.1,123.0,112.0,24.7,False,Lebron James
1,2003-2004,2003-10-30,28.5,0.523,0.500,14.9,33.9,89.0,97.0,14.7,False,Lebron James
2,2003-2004,2003-11-01,18.2,0.311,0.250,6.6,23.6,74.0,112.0,5.0,False,Lebron James
3,2003-2004,2003-11-05,13.7,0.306,0.273,13.1,27.8,85.0,95.0,11.2,False,Lebron James
4,2003-2004,2003-11-07,30.7,0.546,0.472,6.3,12.0,85.0,112.0,9.0,False,Lebron James
...,...,...,...,...,...,...,...,...,...,...,...,...
1698,2023-2024,2023-11-29,31.8,0.590,0.526,15.2,16.8,125.0,107.0,19.6,False,Lebron James
1699,2023-2024,2023-11-30,23.8,0.763,0.750,18.5,31.1,116.0,128.0,19.9,False,Lebron James
1701,2023-2024,2023-12-02,29.9,0.417,0.412,5.3,31.1,77.0,101.0,8.5,False,Lebron James
1702,2023-2024,2023-12-05,29.0,0.561,0.520,10.7,55.6,129.0,100.0,31.0,False,Lebron James


In [441]:
#Lebron James Playoff Advanced Stats
dates = []
usages = []
true_shoot = []
efg = []
trb = []
ast_pct = []
ortg = []
drtg = []
gmsc = []
playoffs = []  
url = 'https://www.basketball-reference.com/players/j/jamesle01/gamelog-playoffs-advanced/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table1 = soup.find('table', id='pgl_advanced_playoffs')
for nba_data in table1.find_all('tbody'):
    rows = nba_data.find_all('tr')
for row in rows:
    date = row.find_all('td', attrs={"data-stat": "date_game"})
    usg = row.find_all('td', attrs={"data-stat": "usg_pct"})
    true = row.find_all('td', attrs={"data-stat": "ts_pct"})
    effect = row.find_all('td', attrs={"data-stat": "efg_pct"})
    board = row.find_all('td', attrs={"data-stat": "trb_pct"})
    ast = row.find_all('td', attrs={"data-stat": "ast_pct"})
    off = row.find_all('td', attrs={"data-stat": "off_rtg"})
    defense = row.find_all('td', attrs={"data-stat": "def_rtg"})
    gs = row.find_all('td', attrs={"data-stat": "game_score"})
    dates.append(date)
    usages.append(usg)
    true_shoot.append(true)
    efg.append(effect)
    trb.append(board)
    ast_pct.append(ast)
    ortg.append(off)
    drtg.append(defense)
    gmsc.append(gs)
    playoffs.append(True)
df = pd.DataFrame({'Date': dates, 'Usage %': usages,'True Shooting %': true_shoot,'Effective Fg %': efg, 'Total Rebound %': trb, 'Assist %' : ast_pct,'Offensive Rating': ortg, 'Defensive Rating' : drtg,'Game Score': gmsc,'Playoffs': playoffs})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Usage %'] = df['Usage %'].str.extract(r'>([\d.]+)<').astype(float)
df['True Shooting %'] = df['True Shooting %'].str.extract(r'>([\d.]+)<').astype(float)
df['Effective Fg %'] = df['Effective Fg %'].str.extract(r'>([\d.]+)<').astype(float)
df['Total Rebound %'] = df['Total Rebound %'].str.extract(r'>([\d.]+)<').astype(float)
df['Assist %'] = df['Assist %'].str.extract(r'>([\d.]+)<').astype(float)
df['Offensive Rating'] = df['Offensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Defensive Rating'] = df['Defensive Rating'].str.extract(r'>([\d.]+)<').astype(float)
df['Game Score'] = df['Game Score'].str.extract(r'>([\d.]+)<').astype(float)
df = df.dropna(subset=['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = (df['Date'].dt.year - 1).astype(int).astype(str) + '-' + df['Date'].dt.year.astype(int).astype(str)
df['Player'] = 'Lebron James'
lbj_play_adv = df

In [442]:
#Lebron James Playoff Basic Stats
own_team = []
game_location = []
opposing_team = []
game_date = []
game_result = []
mins = []
field_goal = []
field_goal_att = []
field_goal_pct = []
three_field_goal = []
three_field_goal_att = []
three_field_goal_pct = []
free_throw = []
free_throw_att = []
free_throw_pct = []
rebounds = []
assist = []
steal = []
block = []
turnover = []
points = []
url = 'https://www.basketball-reference.com/players/j/jamesle01/gamelog-playoffs/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table1 = soup.find('table', id='pgl_basic_playoffs')
for nba_data in table1.find_all('tbody'):
    rows = nba_data.find_all('tr')
for row in rows:
    team = row.find_all('td', attrs={"data-stat": "team_id"})
    location = row.find_all('td', attrs={"data-stat": "game_location"})
    opp_team = row.find_all('td', attrs={"data-stat": "opp_id"})
    date = row.find_all('td', attrs={"data-stat": "date_game"})
    result = row.find_all('td', attrs={"data-stat": "game_result"})
    min = row.find_all('td', attrs={"data-stat": "mp"})
    fg = row.find_all('td', attrs={"data-stat": "fg"})
    fga = row.find_all('td', attrs={"data-stat": "fga"})
    fg_pct = row.find_all('td', attrs={"data-stat": "fg_pct"})
    fg3 = row.find_all('td', attrs={"data-stat": "fg3"})
    fg3a = row.find_all('td', attrs={"data-stat": "fg3a"})
    fg3_pct = row.find_all('td', attrs={"data-stat": "fg3_pct"})
    ft = row.find_all('td', attrs={"data-stat": "ft"})
    fta = row.find_all('td', attrs={"data-stat": "fta"})
    ft_pct = row.find_all('td', attrs={"data-stat": "ft_pct"})
    trb = row.find_all('td', attrs={"data-stat": "trb"})
    ast = row.find_all('td', attrs={"data-stat": "ast"})
    stl = row.find_all('td', attrs={"data-stat": "stl"})
    blk = row.find_all('td', attrs={"data-stat": "blk"})
    tov = row.find_all('td', attrs={"data-stat": "tov"})
    pts = row.find_all('td', attrs={"data-stat": "pts"})
    own_team.append(team)
    game_location.append(location)
    opposing_team.append(opp_team)
    game_date.append(date)
    game_result.append(result)
    mins.append(min)
    field_goal.append(fg)
    field_goal_att.append(fga)
    field_goal_pct.append(fg_pct)
    three_field_goal.append(fg3)
    three_field_goal_att.append(fg3a)
    three_field_goal_pct.append(fg3_pct)
    free_throw.append(ft)
    free_throw_att.append(fta)
    free_throw_pct.append(ft_pct)
    rebounds.append(trb)
    assist.append(ast)
    steal.append(stl)
    block.append(blk)
    turnover.append(tov)
    points.append(pts)
    playoffs.append(True)
df = pd.DataFrame({
    'Team': own_team,
    'Location': game_location,
    'Opp Team': opposing_team,
    'Date': game_date,
    'Result': game_result,
    'Mins Played': mins,
    'Field Goal': field_goal,
    'Field Goal Att': field_goal_att,
    'Field Goal %': field_goal_pct,
    '3 Point Field Goal': three_field_goal,
    '3 Point Field Goal Att': three_field_goal_att,
    '3 Point Field Goal %': three_field_goal_pct,
    'Free Throw': free_throw,
    'Free Throw Att': free_throw_att,
    'Free Throw %': free_throw_pct,
    'Rebounds': rebounds,
    'Assists': assist,
    'Steals': steal,
    'Blocks': block,
    'Turnovers': turnover,
    'Points': points
})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Field Goal'] = df['Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal Att'] = df['Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal %'] = df['Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal'] = df['3 Point Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal Att'] = df['3 Point Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal %'] = df['3 Point Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw'] = df['Free Throw'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw Att'] = df['Free Throw Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw %'] = df['Free Throw %'].str.extract(r'>([\d.]+)<').astype(float)
df['Rebounds'] = df['Rebounds'].str.extract(r'>([\d.]+)<').astype(float)
df['Assists'] = df['Assists'].str.extract(r'>([\d.]+)<').astype(float)
df['Steals'] = df['Steals'].str.extract(r'>([\d.]+)<').astype(float)
df['Blocks'] = df['Blocks'].str.extract(r'>([\d.]+)<').astype(float)
df['Turnovers'] = df['Turnovers'].str.extract(r'>([\d.]+)<').astype(float)
df['Points'] = df['Points'].str.extract(r'>([\d.]+)<').astype(float)
df['Team'] = df['Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Location'] = df['Location'].str.extract(r'>([^<>]+)<')
df['Opp Team'] = df['Opp Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Result'] = df['Result'].str.extract(r'>([^<>]+)<')
df['Result'] = df['Result'].astype(str).str.split().str[0]
df['Mins Played'] = df['Mins Played'].str.extract(r'>(\d+:\d+)<')
df = df.dropna(subset=['Date'])
df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = (df['Date'].dt.year - 1).astype(int).astype(str) + '-' + df['Date'].dt.year.astype(int).astype(str)
df['Player'] = 'Lebron James'
lbj_play_basic = df
lbj_play_basic

,Team,Location,Opp Team,Date,Result,Mins Played,Field Goal,Field Goal Att,Field Goal %,3 Point Field Goal,3 Point Field Goal Att,3 Point Field Goal %,Free Throw,Free Throw Att,Free Throw %,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Season,Player
0,CLE,NaN,WAS,2006-04-22,W,48:00,12.0,27.0,0.444,1.0,4.0,0.250,7.0,11.0,0.636,11.0,11.0,0.0,0.0,4.0,32.0,2005-2006,Lebron James
1,CLE,NaN,WAS,2006-04-25,L,43:28,7.0,25.0,0.280,1.0,6.0,0.167,11.0,15.0,0.733,9.0,2.0,2.0,3.0,10.0,26.0,2005-2006,Lebron James
2,CLE,@,WAS,2006-04-28,W,47:37,16.0,28.0,0.571,3.0,5.0,0.600,6.0,9.0,0.667,5.0,3.0,2.0,0.0,4.0,41.0,2005-2006,Lebron James
3,CLE,@,WAS,2006-04-30,L,45:20,13.0,23.0,0.565,7.0,12.0,0.583,5.0,7.0,0.714,6.0,5.0,0.0,0.0,7.0,38.0,2005-2006,Lebron James
4,CLE,NaN,WAS,2006-05-03,W,46:07,14.0,23.0,0.609,0.0,1.0,0.000,17.0,18.0,0.944,7.0,6.0,2.0,0.0,4.0,45.0,2005-2006,Lebron James
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,LAL,NaN,GSW,2023-05-12,W,43:08,10.0,14.0,0.714,2.0,3.0,0.667,8.0,11.0,0.727,9.0,9.0,2.0,1.0,2.0,30.0,2022-2023,Lebron James
330,LAL,@,DEN,2023-05-16,L,39:52,9.0,16.0,0.563,0.0,4.0,0.000,8.0,11.0,0.727,12.0,9.0,0.0,1.0,3.0,26.0,2022-2023,Lebron James
331,LAL,@,DEN,2023-05-18,L,40:08,9.0,19.0,0.474,0.0,6.0,0.000,4.0,4.0,1.000,9.0,10.0,4.0,2.0,3.0,22.0,2022-2023,Lebron James
332,LAL,NaN,DEN,2023-05-20,L,42:57,8.0,19.0,0.421,3.0,9.0,0.333,4.0,7.0,0.571,7.0,12.0,0.0,0.0,2.0,23.0,2022-2023,Lebron James


In [443]:
#Lebron James Regular Season Basic Stats
own_team = []
game_location = []
opposing_team = []
game_date = []
game_result = []
mins = []
field_goal = []
field_goal_att = []
field_goal_pct = []
three_field_goal = []
three_field_goal_att = []
three_field_goal_pct = []
free_throw = []
free_throw_att = []
free_throw_pct = []
rebounds = []
assist = []
steal = []
block = []
turnover = []
points = []
season = []
for year in range(2004,2025):
    url = f'https://www.basketball-reference.com/players/j/jamesle01/gamelog/{year}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table1 = soup.find('table', id='pgl_basic')
    for nba_data in table1.find_all('tbody'):
        rows = nba_data.find_all('tr')
    for row in rows:
        team = row.find_all('td', attrs={"data-stat": "team_id"})
        location = row.find_all('td', attrs={"data-stat": "game_location"})
        opp_team = row.find_all('td', attrs={"data-stat": "opp_id"})
        date = row.find_all('td', attrs={"data-stat": "date_game"})
        result = row.find_all('td', attrs={"data-stat": "game_result"})
        min = row.find_all('td', attrs={"data-stat": "mp"})
        fg = row.find_all('td', attrs={"data-stat": "fg"})
        fga = row.find_all('td', attrs={"data-stat": "fga"})
        fg_pct = row.find_all('td', attrs={"data-stat": "fg_pct"})
        fg3 = row.find_all('td', attrs={"data-stat": "fg3"})
        fg3a = row.find_all('td', attrs={"data-stat": "fg3a"})
        fg3_pct = row.find_all('td', attrs={"data-stat": "fg3_pct"})
        ft = row.find_all('td', attrs={"data-stat": "ft"})
        fta = row.find_all('td', attrs={"data-stat": "fta"})
        ft_pct = row.find_all('td', attrs={"data-stat": "ft_pct"})
        trb = row.find_all('td', attrs={"data-stat": "trb"})
        ast = row.find_all('td', attrs={"data-stat": "ast"})
        stl = row.find_all('td', attrs={"data-stat": "stl"})
        blk = row.find_all('td', attrs={"data-stat": "blk"})
        tov = row.find_all('td', attrs={"data-stat": "tov"})
        pts = row.find_all('td', attrs={"data-stat": "pts"})
        own_team.append(team)
        game_location.append(location)
        opposing_team.append(opp_team)
        game_date.append(date)
        game_result.append(result)
        mins.append(min)
        field_goal.append(fg)
        field_goal_att.append(fga)
        field_goal_pct.append(fg_pct)
        three_field_goal.append(fg3)
        three_field_goal_att.append(fg3a)
        three_field_goal_pct.append(fg3_pct)
        free_throw.append(ft)
        free_throw_att.append(fta)
        free_throw_pct.append(ft_pct)
        rebounds.append(trb)
        assist.append(ast)
        steal.append(stl)
        block.append(blk)
        turnover.append(tov)
        points.append(pts)
        playoffs.append(False)
        season.append(f'{year-1}-{year}')
df = pd.DataFrame({
    'Season': season,
    'Team': own_team,
    'Location': game_location,
    'Opp Team': opposing_team,
    'Date': game_date,
    'Result': game_result,
    'Mins Played': mins,
    'Field Goal': field_goal,
    'Field Goal Att': field_goal_att,
    'Field Goal %': field_goal_pct,
    '3 Point Field Goal': three_field_goal,
    '3 Point Field Goal Att': three_field_goal_att,
    '3 Point Field Goal %': three_field_goal_pct,
    'Free Throw': free_throw,
    'Free Throw Att': free_throw_att,
    'Free Throw %': free_throw_pct,
    'Rebounds': rebounds,
    'Assists': assist,
    'Steals': steal,
    'Blocks': block,
    'Turnovers': turnover,
    'Points': points
})
df = df.astype(str)
df['Date'] = df['Date'].str.extract(r'>(\d{4}-\d{2}-\d{2})<')
df['Field Goal'] = df['Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal Att'] = df['Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Field Goal %'] = df['Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal'] = df['3 Point Field Goal'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal Att'] = df['3 Point Field Goal Att'].str.extract(r'>([\d.]+)<').astype(float)
df['3 Point Field Goal %'] = df['3 Point Field Goal %'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw'] = df['Free Throw'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw Att'] = df['Free Throw Att'].str.extract(r'>([\d.]+)<').astype(float)
df['Free Throw %'] = df['Free Throw %'].str.extract(r'>([\d.]+)<').astype(float)
df['Rebounds'] = df['Rebounds'].str.extract(r'>([\d.]+)<').astype(float)
df['Assists'] = df['Assists'].str.extract(r'>([\d.]+)<').astype(float)
df['Steals'] = df['Steals'].str.extract(r'>([\d.]+)<').astype(float)
df['Blocks'] = df['Blocks'].str.extract(r'>([\d.]+)<').astype(float)
df['Turnovers'] = df['Turnovers'].str.extract(r'>([\d.]+)<').astype(float)
df['Points'] = df['Points'].str.extract(r'>([\d.]+)<').astype(float)
df['Team'] = df['Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Location'] = df['Location'].str.extract(r'>([^<>]+)<')
df['Opp Team'] = df['Opp Team'].str.extract(r'\/teams\/(\w+)\/\d+.html')
df['Result'] = df['Result'].str.extract(r'>([^<>]+)<')
df['Result'] = df['Result'].astype(str).str.split().str[0]
df['Mins Played'] = df['Mins Played'].str.extract(r'>(\d+:\d+)<')
df = df.dropna(subset=['Date'])
df['Player'] = 'Lebron James'
lbj_reg_basic = df

In [446]:
lbj_playoffs = pd.merge(lbj_play_basic, lbj_play_adv, on=['Date', 'Player', 'Season'])
lbj_reg_season = pd.merge(lbj_reg_basic, lbj_reg_adv, on=['Date', 'Player','Season'])
lbj = pd.concat([lbj_reg_season, lbj_playoffs], ignore_index=True)
lbj['Date'] = pd.to_datetime(lbj['Date'])
player_column = lbj.pop('Player')
lbj.insert(0, 'Player', player_column)

In [447]:
goat = pd.concat([lbj, mj], ignore_index=True)
goat.iloc[:, 2:] = goat.iloc[:, 2:].fillna(0)
goat

,Player,Season,Team,Location,Opp Team,Date,Result,Mins Played,Field Goal,Field Goal Att,Field Goal %,3 Point Field Goal,3 Point Field Goal Att,3 Point Field Goal %,Free Throw,Free Throw Att,Free Throw %,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Usage %,True Shooting %,Effective Fg %,Total Rebound %,Assist %,Offensive Rating,Defensive Rating,Game Score,Playoffs
0,Lebron James,2003-2004,CLE,@,SAC,2003-10-29,L,42:50,12.0,20.0,0.600,0.0,2.0,0.000,1.0,3.0,0.333,6.0,9.0,4.0,0.0,2.0,25.0,25.6,0.586,0.600,8.6,39.1,123.0,112.0,24.7,False
1,Lebron James,2003-2004,CLE,@,PHO,2003-10-30,L,40:21,8.0,17.0,0.471,1.0,5.0,0.200,4.0,7.0,0.571,12.0,8.0,1.0,0.0,7.0,21.0,28.5,0.523,0.500,14.9,33.9,89.0,97.0,14.7,False
2,Lebron James,2003-2004,CLE,@,POR,2003-11-01,L,39:10,3.0,12.0,0.250,0.0,1.0,0.000,2.0,2.0,1.000,4.0,6.0,2.0,0.0,2.0,8.0,18.2,0.311,0.250,6.6,23.6,74.0,112.0,5.0,False
3,Lebron James,2003-2004,CLE,0,DEN,2003-11-05,L,41:06,3.0,11.0,0.273,0.0,2.0,0.000,1.0,1.0,1.000,11.0,7.0,2.0,3.0,2.0,7.0,13.7,0.306,0.273,13.1,27.8,85.0,95.0,11.2,False
4,Lebron James,2003-2004,CLE,@,IND,2003-11-07,L,43:44,8.0,18.0,0.444,1.0,2.0,0.500,6.0,7.0,0.857,5.0,3.0,0.0,0.0,7.0,23.0,30.7,0.546,0.472,6.3,12.0,85.0,112.0,9.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,Michael Jordan,1997-1998,CHI,@,UTA,1998-06-05,W,40:14,14.0,33.0,0.424,0.0,2.0,0.000,9.0,10.0,0.900,5.0,3.0,1.0,1.0,0.0,37.0,45.4,0.495,0.424,8.1,20.9,122.0,113.0,24.8,True
3169,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-07,W,32:06,7.0,14.0,0.500,0.0,0.0,0.000,10.0,11.0,0.909,3.0,2.0,1.0,1.0,2.0,24.0,29.9,0.637,0.500,5.1,11.3,126.0,65.0,18.6,True
3170,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-10,W,43:10,12.0,27.0,0.444,0.0,0.0,0.000,10.0,15.0,0.667,8.0,2.0,2.0,0.0,3.0,34.0,41.0,0.506,0.444,10.6,16.4,108.0,101.0,21.5,True
3171,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-12,L,44:43,9.0,26.0,0.346,0.0,2.0,0.000,10.0,11.0,0.909,4.0,4.0,3.0,0.0,4.0,28.0,37.2,0.454,0.346,6.1,22.0,97.0,104.0,14.8,True


In [459]:
target_values = ['1990-1991', '1991-1992', '1992-1993','1995-1996','1996-1997','1997-1998', '2011-2012', '2012-2013', '2015-2016', '2019-2020']

# Create a new column 'is_target_season' and set True for rows where 'season' is in target_values
goat['Championship Season'] = goat['Season'].isin(target_values)

In [460]:
goat

,Player,Season,Team,Location,Opp Team,Date,Result,Mins Played,Field Goal,Field Goal Att,Field Goal %,3 Point Field Goal,3 Point Field Goal Att,3 Point Field Goal %,Free Throw,Free Throw Att,Free Throw %,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Usage %,True Shooting %,Effective Fg %,Total Rebound %,Assist %,Offensive Rating,Defensive Rating,Game Score,Playoffs,Championship Season
0,Lebron James,2003-2004,CLE,@,SAC,2003-10-29,L,42:50,12.0,20.0,0.600,0.0,2.0,0.000,1.0,3.0,0.333,6.0,9.0,4.0,0.0,2.0,25.0,25.6,0.586,0.600,8.6,39.1,123.0,112.0,24.7,False,False
1,Lebron James,2003-2004,CLE,@,PHO,2003-10-30,L,40:21,8.0,17.0,0.471,1.0,5.0,0.200,4.0,7.0,0.571,12.0,8.0,1.0,0.0,7.0,21.0,28.5,0.523,0.500,14.9,33.9,89.0,97.0,14.7,False,False
2,Lebron James,2003-2004,CLE,@,POR,2003-11-01,L,39:10,3.0,12.0,0.250,0.0,1.0,0.000,2.0,2.0,1.000,4.0,6.0,2.0,0.0,2.0,8.0,18.2,0.311,0.250,6.6,23.6,74.0,112.0,5.0,False,False
3,Lebron James,2003-2004,CLE,0,DEN,2003-11-05,L,41:06,3.0,11.0,0.273,0.0,2.0,0.000,1.0,1.0,1.000,11.0,7.0,2.0,3.0,2.0,7.0,13.7,0.306,0.273,13.1,27.8,85.0,95.0,11.2,False,False
4,Lebron James,2003-2004,CLE,@,IND,2003-11-07,L,43:44,8.0,18.0,0.444,1.0,2.0,0.500,6.0,7.0,0.857,5.0,3.0,0.0,0.0,7.0,23.0,30.7,0.546,0.472,6.3,12.0,85.0,112.0,9.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3168,Michael Jordan,1997-1998,CHI,@,UTA,1998-06-05,W,40:14,14.0,33.0,0.424,0.0,2.0,0.000,9.0,10.0,0.900,5.0,3.0,1.0,1.0,0.0,37.0,45.4,0.495,0.424,8.1,20.9,122.0,113.0,24.8,True,True
3169,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-07,W,32:06,7.0,14.0,0.500,0.0,0.0,0.000,10.0,11.0,0.909,3.0,2.0,1.0,1.0,2.0,24.0,29.9,0.637,0.500,5.1,11.3,126.0,65.0,18.6,True,True
3170,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-10,W,43:10,12.0,27.0,0.444,0.0,0.0,0.000,10.0,15.0,0.667,8.0,2.0,2.0,0.0,3.0,34.0,41.0,0.506,0.444,10.6,16.4,108.0,101.0,21.5,True,True
3171,Michael Jordan,1997-1998,CHI,0,UTA,1998-06-12,L,44:43,9.0,26.0,0.346,0.0,2.0,0.000,10.0,11.0,0.909,4.0,4.0,3.0,0.0,4.0,28.0,37.2,0.454,0.346,6.1,22.0,97.0,104.0,14.8,True,True


In [456]:
goat.iloc[3171,1]

'1997-1998'

In [461]:
goat.to_csv('goat.csv', index=False)